## Esercizio Dati Bancari

In [2]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext

In [5]:
rdd = sc.textFile('movimenti_bancari.csv')

In [6]:
rdd.count()

41

In [12]:
# Estrai la prima riga (intestazione)
header = rdd.first()

# Filtra via l'intestazione
rdd2 = rdd.filter(lambda row: row != header)

Conteggio i record suddivisi per categoria. Creo una funzione che prende una riga e crea una tupla con cat e importo (CIN, 123.0)

In [7]:
def f1(row):
    tok = row.split(',')
    return (tok[3], float(tok[2]))

### rdd3 = rdd2.map(f1)
rdd3.take(2)

In [14]:
rdd3.countByKey()

defaultdict(int, {'CIN': 9, 'COUT': 10, 'BOUT': 10, 'BIN': 11})

**Saldo per mese per conto** 

Serve una nuova funzione


In [17]:
from datetime import datetime as dt
def f2(row):
    tok = row.split(',')
    mm = dt.strptime(tok[0], '%Y-%m-%d').month
    num_conto = tok[1]
    importo = float(tok[2])
    return ((mm, num_conto), importo)
    

In [18]:
rdd4 = rdd2.map(f2)
rdd4.take(2)

[((1, '1003'), 2321.06), ((1, '1004'), -5996.37)]

la somma per ogni mese, per ogni conto

In [25]:
saldo_mese = rdd4.reduceByKey(lambda a,b : a+b)
saldo_mese.collect()

[((1, '1003'), 2321.06),
 ((1, '1004'), -5996.37),
 ((1, '1005'), -7718.04),
 ((1, '1002'), 3079.6),
 ((2, '1004'), -5944.24),
 ((2, '1001'), 1220.3999999999996),
 ((3, '1005'), -8900.0),
 ((3, '1004'), -5839.11),
 ((3, '1002'), -759.59),
 ((4, '1004'), 1341.11),
 ((4, '1001'), 665.49),
 ((5, '1003'), -6272.51),
 ((5, '1004'), 272.18),
 ((6, '1002'), -8037.69),
 ((6, '1003'), -1028.71),
 ((6, '1004'), 5423.0),
 ((7, '1004'), -9036.82),
 ((7, '1002'), 6436.84),
 ((7, '1003'), -663.79),
 ((7, '1001'), 7525.13),
 ((8, '1001'), -865.25),
 ((9, '1002'), -1025.09),
 ((9, '1004'), -8616.98),
 ((9, '1001'), 968.7),
 ((10, '1002'), -872.73),
 ((10, '1001'), -4376.05),
 ((11, '1002'), 8737.210000000001),
 ((11, '1003'), -3922.4300000000003),
 ((12, '1001'), 9687.24),
 ((12, '1005'), 8637.24)]

se voglio limitarla a u nsolo conto


In [26]:
saldo_mese.filter(lambda r : r[0][1] == '1001').sortByKey().collect()

[((2, '1001'), 1220.3999999999996),
 ((4, '1001'), 665.49),
 ((7, '1001'), 7525.13),
 ((8, '1001'), -865.25),
 ((9, '1001'), 968.7),
 ((10, '1001'), -4376.05),
 ((12, '1001'), 9687.24)]